# Pod transcript creator
## Overview
- Scrape news from Gnews (Unoffical api)
- Combined into 1 text
- Prompt transcript creation


In [2]:
import os
import gnews
import openai
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from datetime import datetime
import requests
import datetime
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")
weather_api_key = os.getenv("WEATHER_API_KEY")

In [3]:
def gen_article_text(topic, env="prod"):
    article_titles = list()
    article_body = list()
    if(env == "prod"):
        # google_news = gnews.GNews(period='7d', max_results=10)
        google_news = gnews.GNews(language='en', country='Sweden', period='7d', max_results=10)
        news_response = google_news.get_news(topic) # En lista av artikelobjekt, article.text, article.title
        for news in news_response:
            full_article = google_news.get_full_article(news['url'])
            if(full_article and (full_article.title and full_article.text)):
                if(len(article_body) < 3):
                    article_titles.append(full_article.title)
                    article_body.append(full_article.text)
        with open('./tmp/articles.txt', 'w') as f:   
            for i in range(len(article_titles)):
                if(i != 0):
                    f.write(f"********\n")
                f.write(f"{article_titles[i]}\n")
                f.write(f"********\n")
                f.write(f"{article_body[i]}\n")
    elif(env == "test"):
        with open('./tmp/articles.txt') as f:
            lines = f.readlines()
            lines = ''.join([line.strip() for line in lines])
            lines = lines.split("********")
            for i in range(len(lines)):
                line = lines[i]
                if(i % 2 == 0):
                    article_titles.append(line)
                else:
                    article_body.append(line)
    # print(article_titles)
    # print(article_body)
    combined_text = ""
    for article in article_body:
        combined_text += article
    
    return combined_text
# gen_article_text("Israel and palestine")

Get weather data

In [4]:
def get_weather(city):
    # url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_api_key}&units=metric'
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid=8368401dc9a8d9bee4eeafa222ec5047&units=metric'
    res = requests.get(url)
    data = res.json()
    humidity = data['main']['humidity']
    pressure = data['main']['pressure']
    wind = data['wind']['speed']
    description = data['weather'][0]['description']
    temp = int(data['main']['temp'])

    # print('Temperature:',temp,'°C')
    # print('Wind:',wind)
    # print('Pressure: ',pressure)
    # print('Humidity: ',humidity)
    # print('Description:',description)
    return temp, description

# 

In [5]:
def themes_and_sumamry(text):  
  chat_model = ChatOpenAI(model_name='gpt-3.5-turbo-16k') # Must have set API key as env var
  prompt = f"Identify three key themes in the following text an give each theme an informative title. Return each key theme on a new line. This is the text: {text}"
  themes = chat_model.predict(prompt)

  # prompt = f"Based on these three themes {themes} found in this text {text} write a coherent three segment podcast text for a podcaster to read. The listener have already been listening to an introduction so jump right into the segments without introducing them."
  # podcast_transcript = chat_model.predict(prompt)
  
  prompt = f"Based on these three themes {themes} found in this text {text} write a podcast text for a podcaster to read only for theme 1. The listener have already been listening to an introduction of the themes so no introduction is needed. Only generate the text that will be read for theme 1. Dont introduce the podcast or mention the other themes."
  theme_1 = chat_model.predict(prompt)
  prompt = f"Based on these three themes {themes} found in this text {text} write a podcast text for a podcaster to read only for theme 2. The listener have already been listening to an introduction of the themes so no introduction is needed. Only generate the text that will be read for theme 2. Dont introduce the podcast or mention the other themes."
  theme_2 = chat_model.predict(prompt)
  prompt = f"Based on these three themes {themes} found in this text {text} write a podcast text for a podcaster to read only for theme 3. The listener have already been listening to an introduction of the themes so no introduction is needed. Only generate the text that will be read for theme 3. Dont introduce the podcast or mention the other themes."
  theme_3 = chat_model.predict(prompt)
  # print(themes)
  # print(podcast_transcript)
  return themes, theme_1 + theme_2 + theme_3



In [6]:
def gen_intro(town, day, time, w_temp, w_desc, topic, themes):
    # Connect to OpenAI, prompt ChatGPT to summarize the text
    chat_model = ChatOpenAI(model_name='gpt-3.5-turbo-16k') # Must have set API key as env var
    prompt = f"You are going to generate a podcast introduction for the daily news podcast PodPerfect hosted by Zoe. Begin with setting the scene with how the current day is looking. It is {day} and the time is {time} in {town}. The weather is {w_temp} with {w_desc}. Then introduce the {topic}. The three segments in today’s podcast are {themes}. In your response, only include the words Zoe should read and no other text. Make it around 300 words. Begin with “You are listening to PodPerfect – a podcast with me, Zoe, as your AI host.” and end with “Let’s jump right in!”."
    intro = chat_model.predict(prompt)

    prompt = f"You are going to generate a podcast outro for the daily news podcast PodPerfect hosted by Zoe. The three segments in today’s podcast was the following themes {themes}. In your response, only include the words Zoe should read and no other text. Make it around 300 words. End with “You have been listening to PodPerfect – a podcast with me, Zoe your AI host.”"
    outro = chat_model.predict(prompt)
    return intro, outro

# gen_intro("Stockholm", "Friday", "9:00", "10 degrees", "Israel and palestine conflict","Gaza residents told to move", "US response", "UN says 400 children dead")


In [7]:
town = "Stockholm"
days = ["Monday", "Tuesday", "Wednesday", "Thirsday", "Friday", "Saturday", "Sunday"]
day = days[datetime.datetime.today().weekday()]
time = "9 a.m"
topic = "US politics"
text = gen_article_text(topic, "prod")
themes, podcast_transcript = themes_and_sumamry(text)
w_temp, w_desc = get_weather(town)
intro, outro = gen_intro(town, day, time, w_temp, w_desc, topic, themes)

full_text = intro + "\n" + podcast_transcript + "\n" + outro
print(full_text)


10/17/2023 05:15:16 PM - Article `download()` failed with 429 Client Error: Unknown Error for url: https://thehill.com/opinion/congress-blog/4256438-if-you-needed-more-proof-that-american-politics-is-in-trouble-here-it-is/ on URL https://consent.google.com/m?continue=https://news.google.com/rss/articles/CBMie2h0dHBzOi8vdGhlaGlsbC5jb20vb3Bpbmlvbi9jb25ncmVzcy1ibG9nLzQyNTY0MzgtaWYteW91LW5lZWRlZC1tb3JlLXByb29mLXRoYXQtYW1lcmljYW4tcG9saXRpY3MtaXMtaW4tdHJvdWJsZS1oZXJlLWl0LWlzL9IBf2h0dHBzOi8vdGhlaGlsbC5jb20vb3Bpbmlvbi9jb25ncmVzcy1ibG9nLzQyNTY0MzgtaWYteW91LW5lZWRlZC1tb3JlLXByb29mLXRoYXQtYW1lcmljYW4tcG9saXRpY3MtaXMtaW4tdHJvdWJsZS1oZXJlLWl0LWlzL2FtcC8?oc%3D5&gl=SE&m=0&pc=n&cm=2&hl=en-US&src=1


You are listening to PodPerfect – a podcast with me, Zoe, as your AI host. 

Good morning, listeners! It's Tuesday, 9 a.m here in Stockholm, and the weather outside is a bit gloomy. The temperature is currently at 9 degrees with light rain. So, grab your umbrella and cozy up as we dive into today's news.

First up, we'll be exploring the ever-evolving landscape of U.S. politics. It's no secret that America has become increasingly polarized in recent years. Our first segment will delve into this phenomenon, examining the root causes of political polarization and its impact on American society.

Moving on, we'll be discussing the fascinating overlap in voter opinions on hot-button issues. Despite political polarization, there are often surprising areas of agreement among voters. We'll explore how differing perspectives can converge on topics such as healthcare, climate change, and immigration. It's a complex and nuanced look at the diverse opinions that shape the American political lands